In [21]:
# Setting working directory
import pyodbc
import pandas as pd
import re
import feather
import os
import tqdm
import numpy as np

In [22]:
server = 'onekorpmsdb001sqlkrprod.database.windows.net'
database = 'ONEKORPMSDB001db'

username = 'Jaekwon.lee'
password = 'abc12345!'
driver= '{ODBC Driver 17 for SQL Server}'
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [23]:
# GS
query_volume = "SELECT YYYYMMDD, POC_CODENAME, SKU_CODENAME, SKU_VOL FROM TBF_GS_DAILYSALE WHERE (YYYYMMDD >= 20211101)"
results = pd.read_sql_query(query_volume, cnxn)

results.head()

,YYYYMMDD,POC_CODENAME,SKU_CODENAME,SKU_VOL
0,20211101,V2A52-GS25대치삼성타운점,8801858011024-오비)카스캔500ML,4
1,20211101,V1049-GS25강남동원점,8801119740014-하이트진로)테라캔355ML,1
2,20211101,V1049-GS25강남동원점,8801858011024-오비)카스캔500ML,1
3,20211101,V1049-GS25강남동원점,8801021105833-오비)한맥캔500ML,1
4,20211101,V1049-GS25강남동원점,8801119740113-하이트진로)테라캔500ML,2


In [24]:
results['YYYY'] = results['YYYYMMDD'].apply(lambda x: x[:4])
results['MM'] = results['YYYYMMDD'].apply(lambda x: x[4:6])
results['YYYYMM'] = results['YYYYMMDD'].apply(lambda x: x[:6])

results.head()

,YYYYMMDD,POC_CODENAME,SKU_CODENAME,SKU_VOL,YYYY,MM,YYYYMM
0,20211101,V2A52-GS25대치삼성타운점,8801858011024-오비)카스캔500ML,4,2021,11,202111
1,20211101,V1049-GS25강남동원점,8801119740014-하이트진로)테라캔355ML,1,2021,11,202111
2,20211101,V1049-GS25강남동원점,8801858011024-오비)카스캔500ML,1,2021,11,202111
3,20211101,V1049-GS25강남동원점,8801021105833-오비)한맥캔500ML,1,2021,11,202111
4,20211101,V1049-GS25강남동원점,8801119740113-하이트진로)테라캔500ML,2,2021,11,202111


In [25]:
results = pd.DataFrame(results.groupby(['POC_CODENAME', 'SKU_CODENAME', 'YYYY', 'MM', 'YYYYMM'])['SKU_VOL'].sum()).reset_index()
results.head()

,POC_CODENAME,SKU_CODENAME,YYYY,MM,YYYYMM,SKU_VOL
0,V01CK-GS25관악점,3080216031811-하이트)1664블랑캔500ML,2021,11,202111,59
1,V01CK-GS25관악점,3119780251889-하이네켄)데스페라도스캔500ML,2021,11,202111,6
2,V01CK-GS25관악점,4000856000588-신세계)바르슈타이너캔500ML,2021,11,202111,1
3,V01CK-GS25관악점,4002103292876-비어케이)에딩거헤페캔500ML,2021,11,202111,6
4,V01CK-GS25관악점,4054500121075-신세계)트롤브루레몬캔500ML,2021,11,202111,2


# 여기서 부터 시작

In [26]:
# Distribution : POC 갯수 계산
# Market share / Volume : Segment 계산
query_sku = "SELECT MASTER_CD, MLIQUOR_TYPE_CD, CORE_DETAIL_NM, NS_BRAND_NM, NS_PACKAGE_NM, SIZE, VOL_SKU FROM TBD_GS25_SKUMAPPING"

result_sku = pd.read_sql_query(query_sku, cnxn)
result_sku.head()

query_poc = "SELECT FULL_NAME, GRADE, MPOCS_CD, MNG_YN, BRANCH, TEAM, SR, NEW_EXISTED, AREA, SIDO, REGION FROM TBD_GSPOC_INFO"

result_poc = pd.read_sql_query(query_poc, cnxn)
result_poc.head()

,FULL_NAME,GRADE,MPOCS_CD,MNG_YN,BRANCH,TEAM,SR,NEW_EXISTED,AREA,SIDO,REGION
0,VP231-GS25성북마을점,B,2000337566,Y,특수강북팀,나르샤,M성북동,신규,LDA,서울특별시,Seoul
1,VP951-GS25장항신성점,A,2000280845,Y,의정부,카스천하,L일산동,신규,LDA,경기도,Sudo_KW
2,V2K89-GS25인계골드점,A,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,LDA,경기도,Sudo_KW
3,VT748-GS25인계원룸점,B,2000359192,Y,수원,수호천사,L탑동,기존,LDA,경기도,Sudo_KW
4,VT748-GS25인계원룸점,B,2000359192,Y,수원,수호천사,L탑동,기존,LDA,경기도,Sudo_KW


In [27]:
results_master = pd.merge(results, result_sku, left_on='SKU_CODENAME', right_on='MASTER_CD', how='left')
results_master = pd.merge(results_master, result_poc, left_on='POC_CODENAME', right_on='FULL_NAME', how='left')
results_master

,POC_CODENAME,SKU_CODENAME,YYYY,MM,YYYYMM,SKU_VOL,MASTER_CD,MLIQUOR_TYPE_CD,CORE_DETAIL_NM,NS_BRAND_NM,...,GRADE,MPOCS_CD,MNG_YN,BRANCH,TEAM,SR,NEW_EXISTED,AREA,SIDO,REGION
0,V01CK-GS25관악점,3080216031811-하이트)1664블랑캔500ML,2021,11,202111,59,3080216031811-하이트)1664블랑캔500ML,100,Premium & Above,KRONENBOURG 1664,...,A,7000036023,Y,특수강남팀,블랙캣,M신림동,기존,LDA,서울특별시,Seoul
1,V01CK-GS25관악점,3119780251889-하이네켄)데스페라도스캔500ML,2021,11,202111,6,3119780251889-하이네켄)데스페라도스캔500ML,100,Premium & Above,DESPERADOS,...,A,7000036023,Y,특수강남팀,블랙캣,M신림동,기존,LDA,서울특별시,Seoul
2,V01CK-GS25관악점,4000856000588-신세계)바르슈타이너캔500ML,2021,11,202111,1,4000856000588-신세계)바르슈타이너캔500ML,100,Premium & Above,WARSTEINER,...,A,7000036023,Y,특수강남팀,블랙캣,M신림동,기존,LDA,서울특별시,Seoul
3,V01CK-GS25관악점,4002103292876-비어케이)에딩거헤페캔500ML,2021,11,202111,6,4002103292876-비어케이)에딩거헤페캔500ML,100,Premium & Above,ERDINGER,...,A,7000036023,Y,특수강남팀,블랙캣,M신림동,기존,LDA,서울특별시,Seoul
4,V01CK-GS25관악점,4054500121075-신세계)트롤브루레몬캔500ML,2021,11,202111,2,4054500121075-신세계)트롤브루레몬캔500ML,400,Other Alcohol,FLAVORED_OTHERS,...,A,7000036023,Y,특수강남팀,블랙캣,M신림동,기존,LDA,서울특별시,Seoul
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1120831,VY999-GS25산청금서점,8801858041984-오비)카스큐팩1.0L,2021,11,202111,3,8801858041984-오비)카스큐팩1.0L,100,CORE,CASS FRESH,...,D,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Residential,경상남도,East
1120832,VY999-GS25산청금서점,8801858045258-오비)카스제로캔355ML,2021,11,202111,1,8801858045258-오비)카스제로캔355ML,500,Non-Alcohol,CASS ZERO,...,D,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Residential,경상남도,East
1120833,VY999-GS25산청금서점,8801858133337-오비)카스큐팩1.6L,2021,11,202111,7,8801858133337-오비)카스큐팩1.6L,100,CORE,CASS FRESH,...,D,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Residential,경상남도,East
1120834,VY999-GS25산청금서점,8809556110933-제주)금성맥주캔500ML,2021,11,202111,1,8809556110933-제주)금성맥주캔500ML,100,Premium & Above,JEJU,...,D,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Residential,경상남도,East


In [28]:
results_master = results_master.query("MLIQUOR_TYPE_CD == '100'")

In [29]:
total_master = pd.DataFrame(results_master.groupby(['YYYY', 'MM', 'YYYYMM', 'REGION', 'SIDO', 'GRADE', 'MNG_YN', 'BRANCH', 'TEAM', 'SR', 'NEW_EXISTED', 'AREA'])['POC_CODENAME'].nunique()).reset_index()
total_master.rename(columns = {'POC_CODENAME' : 'Total_POC'}, inplace=True)
total_master.head()

,YYYY,MM,YYYYMM,REGION,SIDO,GRADE,MNG_YN,BRANCH,TEAM,SR,NEW_EXISTED,AREA,Total_POC
0,2021,11,202111,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,LDA,6
1,2021,11,202111,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Office,5
2,2021,11,202111,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Residential,6
3,2021,11,202111,East,경상남도,A,Y,울산,챔피언,M삼호동,신규,Residential,1
4,2021,11,202111,East,경상남도,A,Y,울산,챔피언,M신정동,신규,Residential,2


In [30]:
value_master = results_master.query("CORE_DETAIL_NM == 'VALUE'")
value_master

,POC_CODENAME,SKU_CODENAME,YYYY,MM,YYYYMM,SKU_VOL,MASTER_CD,MLIQUOR_TYPE_CD,CORE_DETAIL_NM,NS_BRAND_NM,...,GRADE,MPOCS_CD,MNG_YN,BRANCH,TEAM,SR,NEW_EXISTED,AREA,SIDO,REGION
68,V01CK-GS25관악점,8801119241511-하이트)필라이트후레쉬캔500ML,2021,11,202111,80,8801119241511-하이트)필라이트후레쉬캔500ML,100,VALUE,FILITE,...,A,7000036023,Y,특수강남팀,블랙캣,M신림동,기존,LDA,서울특별시,Seoul
69,V01CK-GS25관악점,8801119241719-하이트)필라이트후레쉬페트1.6L,2021,11,202111,21,8801119241719-하이트)필라이트후레쉬페트1.6L,100,VALUE,FILITE,...,A,7000036023,Y,특수강남팀,블랙캣,M신림동,기존,LDA,서울특별시,Seoul
165,V1049-GS25강남동원점,8801021105192-오비)필굿큐팩1.6L,2021,11,202111,12,8801021105192-오비)필굿큐팩1.6L,100,VALUE,FILGOOD,...,B,7000035714,Y,특수강남팀,피오나,M청담동,기존,Office,서울특별시,Seoul
188,V1049-GS25강남동원점,8801119241511-하이트)필라이트후레쉬캔500ML,2021,11,202111,37,8801119241511-하이트)필라이트후레쉬캔500ML,100,VALUE,FILITE,...,B,7000035714,Y,특수강남팀,피오나,M청담동,기존,Office,서울특별시,Seoul
248,V1052-GS25연대2점,8801021105154-오비)필굿캔500ML,2021,11,202111,7,8801021105154-오비)필굿캔500ML,100,VALUE,FILGOOD,...,C,2000339395,Y,특수강북팀,나르샤,M연희동,신규,LDA,서울특별시,Seoul
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1120711,VY997-GS25진관힐스테이트점,8801021105192-오비)필굿큐팩1.6L,2021,11,202111,10,8801021105192-오비)필굿큐팩1.6L,100,VALUE,FILGOOD,...,B,2000273171,Y,특수강북팀,나르샤,M불광동,신규,Residential,서울특별시,Seoul
1120713,VY997-GS25진관힐스테이트점,8801021105567-오비)필굿SEVEN캔500ML,2021,11,202111,2,8801021105567-오비)필굿SEVEN캔500ML,100,VALUE,FILGOOD,...,B,2000273171,Y,특수강북팀,나르샤,M불광동,신규,Residential,서울특별시,Seoul
1120734,VY997-GS25진관힐스테이트점,8801119241511-하이트)필라이트후레쉬캔500ML,2021,11,202111,32,8801119241511-하이트)필라이트후레쉬캔500ML,100,VALUE,FILITE,...,B,2000273171,Y,특수강북팀,나르샤,M불광동,신규,Residential,서울특별시,Seoul
1120735,VY997-GS25진관힐스테이트점,8801119241719-하이트)필라이트후레쉬페트1.6L,2021,11,202111,4,8801119241719-하이트)필라이트후레쉬페트1.6L,100,VALUE,FILITE,...,B,2000273171,Y,특수강북팀,나르샤,M불광동,신규,Residential,서울특별시,Seoul


In [31]:
# FG_extra : Distribution
FG_extra = value_master.query("SKU_CODENAME == '8801021228099-오비)필굿엑스트라캔500ML'").groupby(['YYYY', 'MM', 'YYYYMM', 'REGION', 'SIDO', 'GRADE', 'MNG_YN', 'BRANCH', 'TEAM', 'SR', 'NEW_EXISTED', 'AREA'])['POC_CODENAME'].nunique().reset_index()
FG_extra.rename(columns = {'POC_CODENAME' : 'FG_extra_dist'}, inplace=True)

total_master = pd.merge(total_master, FG_extra, on=['YYYY', 'MM', 'YYYYMM', 'REGION', 'SIDO', 'GRADE', 'MNG_YN', 'BRANCH', 'TEAM', 'SR', 'NEW_EXISTED', 'AREA'], how='left')
total_master

,YYYY,MM,YYYYMM,REGION,SIDO,GRADE,MNG_YN,BRANCH,TEAM,SR,NEW_EXISTED,AREA,Total_POC,FG_extra_dist
0,2021,11,202111,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,LDA,6,1.0
1,2021,11,202111,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Office,5,5.0
2,2021,11,202111,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Residential,6,2.0
3,2021,11,202111,East,경상남도,A,Y,울산,챔피언,M삼호동,신규,Residential,1,1.0
4,2021,11,202111,East,경상남도,A,Y,울산,챔피언,M신정동,신규,Residential,2,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2893,2021,11,202111,West,충청북도,D,Y,충주,푸른물결,M장락동,기존,Residential,3,2.0
2894,2021,11,202111,West,충청북도,D,Y,충주,푸른물결,M장락동,신규,Residential,15,10.0
2895,2021,11,202111,West,충청북도,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,LDA,4,3.0
2896,2021,11,202111,West,충청북도,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Office,5,3.0


In [32]:
# FG_seven : Distribution
FG_seven = value_master.query("SKU_CODENAME == '8801021105567-오비)필굿SEVEN캔500ML'").groupby(['YYYY', 'MM', 'YYYYMM', 'REGION', 'SIDO', 'GRADE', 'MNG_YN', 'BRANCH', 'TEAM', 'SR', 'NEW_EXISTED', 'AREA'])['POC_CODENAME'].nunique().reset_index()
FG_seven.rename(columns = {'POC_CODENAME' : 'FG_seven_dist'}, inplace=True)


total_master = pd.merge(total_master, FG_seven, on=['YYYY', 'MM', 'YYYYMM', 'REGION', 'SIDO', 'GRADE', 'MNG_YN', 'BRANCH', 'TEAM', 'SR', 'NEW_EXISTED', 'AREA'], how='left')
total_master

,YYYY,MM,YYYYMM,REGION,SIDO,GRADE,MNG_YN,BRANCH,TEAM,SR,NEW_EXISTED,AREA,Total_POC,FG_extra_dist,FG_seven_dist
0,2021,11,202111,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,LDA,6,1.0,1.0
1,2021,11,202111,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Office,5,5.0,1.0
2,2021,11,202111,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Residential,6,2.0,1.0
3,2021,11,202111,East,경상남도,A,Y,울산,챔피언,M삼호동,신규,Residential,1,1.0,1.0
4,2021,11,202111,East,경상남도,A,Y,울산,챔피언,M신정동,신규,Residential,2,2.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2893,2021,11,202111,West,충청북도,D,Y,충주,푸른물결,M장락동,기존,Residential,3,2.0,1.0
2894,2021,11,202111,West,충청북도,D,Y,충주,푸른물결,M장락동,신규,Residential,15,10.0,12.0
2895,2021,11,202111,West,충청북도,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,LDA,4,3.0,2.0
2896,2021,11,202111,West,충청북도,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Office,5,3.0,NaN


In [33]:
# FG_original : Distribution
FG_original = value_master.query("(SKU_CODENAME != '8801021105567-오비)필굿SEVEN캔500ML') and (SKU_CODENAME != '8801021228099-오비)필굿엑스트라캔500ML') and (NS_BRAND_NM == 'FILGOOD')").groupby(['YYYY', 'MM', 'YYYYMM', 'REGION', 'SIDO', 'GRADE', 'MNG_YN', 'BRANCH', 'TEAM', 'SR', 'NEW_EXISTED', 'AREA'])['POC_CODENAME'].nunique().reset_index()
FG_original.rename(columns = {'POC_CODENAME' : 'FG_original_dist'}, inplace=True)


total_master = pd.merge(total_master, FG_original, on=['YYYY', 'MM', 'YYYYMM', 'REGION', 'SIDO', 'GRADE', 'MNG_YN', 'BRANCH', 'TEAM', 'SR', 'NEW_EXISTED', 'AREA'], how='left')
total_master

,YYYY,MM,YYYYMM,REGION,SIDO,GRADE,MNG_YN,BRANCH,TEAM,SR,NEW_EXISTED,AREA,Total_POC,FG_extra_dist,FG_seven_dist,FG_original_dist
0,2021,11,202111,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,LDA,6,1.0,1.0,4.0
1,2021,11,202111,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Office,5,5.0,1.0,2.0
2,2021,11,202111,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Residential,6,2.0,1.0,1.0
3,2021,11,202111,East,경상남도,A,Y,울산,챔피언,M삼호동,신규,Residential,1,1.0,1.0,1.0
4,2021,11,202111,East,경상남도,A,Y,울산,챔피언,M신정동,신규,Residential,2,2.0,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2893,2021,11,202111,West,충청북도,D,Y,충주,푸른물결,M장락동,기존,Residential,3,2.0,1.0,3.0
2894,2021,11,202111,West,충청북도,D,Y,충주,푸른물결,M장락동,신규,Residential,15,10.0,12.0,15.0
2895,2021,11,202111,West,충청북도,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,LDA,4,3.0,2.0,4.0
2896,2021,11,202111,West,충청북도,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Office,5,3.0,NaN,3.0


In [34]:
value_master['Volume'] = value_master['SKU_VOL'] * value_master['VOL_SKU']
value_master.head()

C:\Users\15156\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,POC_CODENAME,SKU_CODENAME,YYYY,MM,YYYYMM,SKU_VOL,MASTER_CD,MLIQUOR_TYPE_CD,CORE_DETAIL_NM,NS_BRAND_NM,...,MPOCS_CD,MNG_YN,BRANCH,TEAM,SR,NEW_EXISTED,AREA,SIDO,REGION,Volume
68,V01CK-GS25관악점,8801119241511-하이트)필라이트후레쉬캔500ML,2021,11,202111,80,8801119241511-하이트)필라이트후레쉬캔500ML,100,VALUE,FILITE,...,7000036023,Y,특수강남팀,블랙캣,M신림동,기존,LDA,서울특별시,Seoul,40000
69,V01CK-GS25관악점,8801119241719-하이트)필라이트후레쉬페트1.6L,2021,11,202111,21,8801119241719-하이트)필라이트후레쉬페트1.6L,100,VALUE,FILITE,...,7000036023,Y,특수강남팀,블랙캣,M신림동,기존,LDA,서울특별시,Seoul,33600
165,V1049-GS25강남동원점,8801021105192-오비)필굿큐팩1.6L,2021,11,202111,12,8801021105192-오비)필굿큐팩1.6L,100,VALUE,FILGOOD,...,7000035714,Y,특수강남팀,피오나,M청담동,기존,Office,서울특별시,Seoul,19200
188,V1049-GS25강남동원점,8801119241511-하이트)필라이트후레쉬캔500ML,2021,11,202111,37,8801119241511-하이트)필라이트후레쉬캔500ML,100,VALUE,FILITE,...,7000035714,Y,특수강남팀,피오나,M청담동,기존,Office,서울특별시,Seoul,18500
248,V1052-GS25연대2점,8801021105154-오비)필굿캔500ML,2021,11,202111,7,8801021105154-오비)필굿캔500ML,100,VALUE,FILGOOD,...,2000339395,Y,특수강북팀,나르샤,M연희동,신규,LDA,서울특별시,Seoul,3500


In [35]:
# Volume : Value
Value_vol = value_master.query("CORE_DETAIL_NM == 'VALUE'").groupby(['YYYY', 'MM', 'YYYYMM', 'REGION', 'SIDO', 'GRADE', 'MNG_YN', 'BRANCH', 'TEAM', 'SR', 'NEW_EXISTED', 'AREA'])['Volume'].sum().reset_index()
Value_vol.rename(columns = {'Volume' : 'Value_volume'}, inplace=True)

total_master = pd.merge(total_master, Value_vol, on=['YYYY', 'MM', 'YYYYMM', 'REGION', 'SIDO', 'GRADE', 'MNG_YN', 'BRANCH', 'TEAM', 'SR', 'NEW_EXISTED', 'AREA'], how='left')
total_master

,YYYY,MM,YYYYMM,REGION,SIDO,GRADE,MNG_YN,BRANCH,TEAM,SR,NEW_EXISTED,AREA,Total_POC,FG_extra_dist,FG_seven_dist,FG_original_dist,Value_volume
0,2021,11,202111,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,LDA,6,1.0,1.0,4.0,388800.0
1,2021,11,202111,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Office,5,5.0,1.0,2.0,152600.0
2,2021,11,202111,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Residential,6,2.0,1.0,1.0,208400.0
3,2021,11,202111,East,경상남도,A,Y,울산,챔피언,M삼호동,신규,Residential,1,1.0,1.0,1.0,106000.0
4,2021,11,202111,East,경상남도,A,Y,울산,챔피언,M신정동,신규,Residential,2,2.0,NaN,2.0,152500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2893,2021,11,202111,West,충청북도,D,Y,충주,푸른물결,M장락동,기존,Residential,3,2.0,1.0,3.0,85300.0
2894,2021,11,202111,West,충청북도,D,Y,충주,푸른물결,M장락동,신규,Residential,15,10.0,12.0,15.0,375400.0
2895,2021,11,202111,West,충청북도,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,LDA,4,3.0,2.0,4.0,206800.0
2896,2021,11,202111,West,충청북도,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Office,5,3.0,NaN,3.0,231000.0


In [36]:
# FG_extra : volume
FG_extra = value_master.query("SKU_CODENAME == '8801021228099-오비)필굿엑스트라캔500ML'").groupby(['YYYY', 'MM', 'YYYYMM', 'REGION', 'SIDO', 'GRADE', 'MNG_YN', 'BRANCH', 'TEAM', 'SR', 'NEW_EXISTED', 'AREA'])['Volume'].sum().reset_index()
FG_extra.rename(columns = {'Volume' : 'FG_extra_volume'}, inplace=True)


total_master = pd.merge(total_master, FG_extra, on=['YYYY', 'MM', 'YYYYMM', 'REGION', 'SIDO', 'GRADE', 'MNG_YN', 'BRANCH', 'TEAM', 'SR', 'NEW_EXISTED', 'AREA'], how='left')

# FG_seven : Volume
FG_seven = value_master.query("SKU_CODENAME == '8801021105567-오비)필굿SEVEN캔500ML'").groupby(['YYYY', 'MM', 'YYYYMM', 'REGION', 'SIDO', 'GRADE', 'MNG_YN', 'BRANCH', 'TEAM', 'SR', 'NEW_EXISTED', 'AREA'])['Volume'].sum().reset_index()
FG_seven.rename(columns = {'Volume' : 'FG_seven_volume'}, inplace=True)


total_master = pd.merge(total_master, FG_seven, on=['YYYY', 'MM', 'YYYYMM', 'REGION', 'SIDO', 'GRADE', 'MNG_YN', 'BRANCH', 'TEAM', 'SR', 'NEW_EXISTED', 'AREA'], how='left')

# FG_original : Volume
FG_original = value_master.query("(SKU_CODENAME != '8801021105567-오비)필굿SEVEN캔500ML') and (SKU_CODENAME != '8801021228099-오비)필굿엑스트라캔500ML') and (NS_BRAND_NM == 'FILGOOD')").groupby(['YYYY', 'MM', 'YYYYMM', 'REGION', 'SIDO', 'GRADE', 'MNG_YN', 'BRANCH', 'TEAM', 'SR', 'NEW_EXISTED', 'AREA'])['Volume'].sum().reset_index()
FG_original.rename(columns = {'Volume' : 'FG_original_volume'}, inplace=True)


total_master = pd.merge(total_master, FG_original, on=['YYYY', 'MM', 'YYYYMM',
                                                       'REGION', 'SIDO', 'GRADE', 'MNG_YN', 'BRANCH', 'TEAM', 'SR', 'NEW_EXISTED', 'AREA'], how='left')
total_master


,YYYY,MM,YYYYMM,REGION,SIDO,GRADE,MNG_YN,BRANCH,TEAM,SR,NEW_EXISTED,AREA,Total_POC,FG_extra_dist,FG_seven_dist,FG_original_dist,Value_volume,FG_extra_volume,FG_seven_volume,FG_original_volume
0,2021,11,202111,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,LDA,6,1.0,1.0,4.0,388800.0,16500.0,1500.0,113600.0
1,2021,11,202111,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Office,5,5.0,1.0,2.0,152600.0,19000.0,1000.0,22000.0
2,2021,11,202111,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Residential,6,2.0,1.0,1.0,208400.0,3000.0,500.0,15500.0
3,2021,11,202111,East,경상남도,A,Y,울산,챔피언,M삼호동,신규,Residential,1,1.0,1.0,1.0,106000.0,14500.0,5000.0,25500.0
4,2021,11,202111,East,경상남도,A,Y,울산,챔피언,M신정동,신규,Residential,2,2.0,NaN,2.0,152500.0,7500.0,NaN,33300.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2893,2021,11,202111,West,충청북도,D,Y,충주,푸른물결,M장락동,기존,Residential,3,2.0,1.0,3.0,85300.0,4500.0,500.0,23100.0
2894,2021,11,202111,West,충청북도,D,Y,충주,푸른물결,M장락동,신규,Residential,15,10.0,12.0,15.0,375400.0,26000.0,30000.0,154400.0
2895,2021,11,202111,West,충청북도,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,LDA,4,3.0,2.0,4.0,206800.0,18000.0,18500.0,114800.0
2896,2021,11,202111,West,충청북도,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Office,5,3.0,NaN,3.0,231000.0,10000.0,NaN,11700.0


In [37]:
total_master.fillna(0, inplace=True)

In [38]:
adhoc_month_original = pd.read_csv("//172.18.169.51/apac-seoulhq-fs/01.기본공용폴더/STRATEGY&TECHNOLOGY/Commercial Strategy & Intelligence/03. Data Models/GS/ad_hoc_fg_month.csv")
adhoc_month_original = adhoc_month_original.query("YYYYMM != 202111")
adhoc_month_original

,YYYY,MM,YYYYMM,REGION,SIDO,GRADE,MNG_YN,BRANCH,TEAM,SR,NEW_EXISTED,AREA,Total_POC,FG_extra_dist,FG_seven_dist,FG_original_dist,Value_volume,FG_extra_volume,FG_seven_volume,FG_original_volume
0,2021,1,202101,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,LDA,5,0.0,2.0,3.0,623300.0,0.0,8500.0,259300.0
1,2021,1,202101,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Office,5,0.0,1.0,1.0,413600.0,0.0,9000.0,74600.0
2,2021,1,202101,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Residential,5,0.0,3.0,2.0,373700.0,0.0,24000.0,44500.0
3,2021,1,202101,East,경상남도,A,Y,울산,챔피언,M삼호동,신규,Residential,1,0.0,1.0,1.0,219700.0,0.0,5500.0,50000.0
4,2021,1,202101,East,경상남도,A,Y,울산,챔피언,M신정동,신규,Residential,2,0.0,1.0,2.0,361900.0,0.0,10500.0,110300.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28826,2021,10,202110,West,충청북도,D,Y,충주,푸른물결,M장락동,기존,Residential,3,1.0,2.0,3.0,200900.0,1000.0,2000.0,78800.0
28827,2021,10,202110,West,충청북도,D,Y,충주,푸른물결,M장락동,신규,Residential,15,11.0,10.0,15.0,949400.0,41000.0,54000.0,463100.0
28828,2021,10,202110,West,충청북도,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,LDA,4,4.0,2.0,4.0,401000.0,19500.0,17000.0,203700.0
28829,2021,10,202110,West,충청북도,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Office,5,3.0,1.0,4.0,609200.0,14500.0,500.0,33900.0


In [39]:
adhoc_month_original = pd.concat([adhoc_month_original, total_master], axis=0)
adhoc_month_original

,YYYY,MM,YYYYMM,REGION,SIDO,GRADE,MNG_YN,BRANCH,TEAM,SR,NEW_EXISTED,AREA,Total_POC,FG_extra_dist,FG_seven_dist,FG_original_dist,Value_volume,FG_extra_volume,FG_seven_volume,FG_original_volume
0,2021,1,202101,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,LDA,5,0.0,2.0,3.0,623300.0,0.0,8500.0,259300.0
1,2021,1,202101,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Office,5,0.0,1.0,1.0,413600.0,0.0,9000.0,74600.0
2,2021,1,202101,East,경상남도,A,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Residential,5,0.0,3.0,2.0,373700.0,0.0,24000.0,44500.0
3,2021,1,202101,East,경상남도,A,Y,울산,챔피언,M삼호동,신규,Residential,1,0.0,1.0,1.0,219700.0,0.0,5500.0,50000.0
4,2021,1,202101,East,경상남도,A,Y,울산,챔피언,M신정동,신규,Residential,2,0.0,1.0,2.0,361900.0,0.0,10500.0,110300.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2893,2021,11,202111,West,충청북도,D,Y,충주,푸른물결,M장락동,기존,Residential,3,2.0,1.0,3.0,85300.0,4500.0,500.0,23100.0
2894,2021,11,202111,West,충청북도,D,Y,충주,푸른물결,M장락동,신규,Residential,15,10.0,12.0,15.0,375400.0,26000.0,30000.0,154400.0
2895,2021,11,202111,West,충청북도,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,LDA,4,3.0,2.0,4.0,206800.0,18000.0,18500.0,114800.0
2896,2021,11,202111,West,충청북도,Not_allocated,N,Not_allocated,Not_allocated,Not_allocated,Not_allocated,Office,5,3.0,0.0,3.0,231000.0,10000.0,0.0,11700.0


In [40]:
adhoc_month_original.groupby('YYYYMM')['Total_POC', 'FG_extra_dist', 'FG_seven_dist', 'FG_original_dist', 'Value_volume', 'FG_extra_volume', 'FG_seven_volume', 'FG_original_volume'].sum()

C:\Users\15156\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,Total_POC,FG_extra_dist,FG_seven_dist,FG_original_dist,Value_volume,FG_extra_volume,FG_seven_volume,FG_original_volume
YYYYMM,,,,,,,,
202101,14185,0.0,7515.0,11929.0,1.589137e+09,0.0,59721000.0,813344000.0
202102,14254,0.0,7033.0,11978.0,1.365529e+09,0.0,48939500.0,700118900.0
202103,14398,0.0,7032.0,12136.0,1.442632e+09,0.0,52393500.0,745706900.0
202104,14497,0.0,7269.0,12217.0,1.385294e+09,0.0,58403500.0,616657500.0
202105,14580,0.0,7246.0,12275.0,1.437676e+09,0.0,59899000.0,624603900.0
202106,14640,0.0,7143.0,12312.0,1.419257e+09,0.0,61046000.0,599412200.0
202107,14683,0.0,6902.0,12351.0,1.576426e+09,0.0,62195000.0,650077100.0
202108,14731,0.0,6628.0,12337.0,1.599514e+09,0.0,52608500.0,658238200.0
202109,14776,0.0,5351.0,12364.0,1.438865e+09,0.0,28227000.0,590335700.0


In [41]:
adhoc_month_original.to_csv("//172.18.169.51/apac-seoulhq-fs/01.기본공용폴더/STRATEGY&TECHNOLOGY/Commercial Strategy & Intelligence/03. Data Models/GS/ad_hoc_fg_month.csv", index=False, encoding='utf-8-sig')